Imports

In [1]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import helpers.evaluate
from helpers.model_list import model_map

#Import models
models = {name: model_class() for name, model_class in model_map.items()}

c:\Users\conno\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load in Data

In [2]:
df = pd.read_csv("diabetes_binary_5050split_health_indicators_BRFSS2015.csv")
X = df.drop(columns=["Diabetes_binary"])
y = df["Diabetes_binary"]
df.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,5.0,30.0,0.0,1.0,4.0,6.0,8.0
1,0.0,1.0,1.0,1.0,26.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,6.0,8.0
2,0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,10.0,0.0,1.0,13.0,6.0,8.0
3,0.0,1.0,1.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,3.0,0.0,3.0,0.0,1.0,11.0,6.0,8.0
4,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,5.0,8.0


Explore Data

In [3]:
#TODO

Clean Data

In [4]:
#TODO

Split Data

In [3]:
#Transform target
le = LabelEncoder()
y = le.fit_transform(y)

#Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8)

#Scale Data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Reassign col names
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)


Train Models

In [6]:


total_metrics = {}

#train each model
for name, model in models.items():
    model.train(X_train_scaled, y_train)
    metrics = model.predict(X_test_scaled, y_test)
    total_metrics[name] = metrics
    helpers.evaluate.print_stats(metrics, name)


Decision Tree  Accuracy Score:  0.6516426777946741
Decision Tree  Precision Score:  0.6519166725713881
Decision Tree  Recalle Score:  0.65074088481805
Decision Tree  F1 Score:  0.6513282480576252
Logistic Regression  Accuracy Score:  0.7459419316051915
Logistic Regression  Precision Score:  0.7380618217916681
Logistic Regression  Recalle Score:  0.7624924850585282
Logistic Regression  F1 Score:  0.7500782744825187
K Nearest Neighbors  Accuracy Score:  0.6996498921384872
K Nearest Neighbors  Precision Score:  0.68982213106486
K Nearest Neighbors  Recalle Score:  0.7255366552321675
K Nearest Neighbors  F1 Score:  0.7072287910648419
SVC  Accuracy Score:  0.7397001096297344
SVC  Precision Score:  0.7168543639621193
SVC  Recalle Score:  0.7923754287937192
SVC  F1 Score:  0.7527253792014513


Report Stats

In [7]:
#Get best metrics
bestModel = ""
bestPerformance = 0
for name, metrics in total_metrics.items():
    if(metrics[0] > bestPerformance):
        bestModel = name
        bestPerformance = metrics[0]

print("Best Model = ", bestModel)
print("Model Accuracy = ", bestPerformance)


Best Model =  Logistic Regression
Model Accuracy =  0.7459419316051915


Hyperparameter Tuning

In [4]:
svc = models["SVC"]
svc.tune_hyperParams(X_test_scaled, y_test)

[I 2025-04-09 13:33:47,443] A new study created in memory with name: SVC
c:\Users\conno\Desktop\Coding\Spring 25\CIS4930-ML-Project\models\svc.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  svc_gamma = trial.suggest_loguniform("gamma", 1e-4, 1e1)
[W 2025-04-09 13:35:47,549] Trial 0 failed with parameters: {'svc_c': 2.395133964593509e-09, 'svc_kernel': 'sigmoid', 'gamma': 0.04090178079914668} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\conno\AppData\Local\Programs\Python\Python310\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "c:\Users\conno\Desktop\Coding\Spring 25\CIS4930-ML-Project\models\svc.py", line 37, in <lambda>
    study.optimize(lambda trial: self.objective(trial, x, y), n_trials=100)
  F

KeyboardInterrupt: 